In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("train.csv")

In [3]:
data.shape

(990, 194)

In [4]:
x_data = data.drop(['species', 'id'], axis=1).values
le = LabelEncoder().fit(data['species'])
y_data = le.transform(data['species'])

In [5]:
scaler = StandardScaler().fit(x_data)
x_data = scaler.transform(x_data)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

In [7]:
x_train.shape

(693, 192)

In [8]:
testing = pd.read_csv("test.csv")
test = testing.drop(['id'], axis=1).values
test = scaler.transform(test)

In [9]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=192)]

In [10]:
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[1024, 512, 256,128, 99],
                                            n_classes=99,
                                            model_dir="/tmp/leaf__model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x125773890>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [11]:
classifier.fit(x=x_train, y=y_train, steps=2000)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


//anaconda/lib/python2.7/site-packages/tensorflow/python/util/deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 2 into /tmp/leaf__model/model.ckpt.
INFO:tensorflow:loss = 4.58914, step = 2
INFO:tensorflow:global_step/sec: 8.5116
INFO:tensorflow:loss = 0.0303625, step = 102
INFO:tensorflow:global_step/sec: 8.14976
INFO:tensorflow:loss = 0.0077215, step = 202
INFO:tensorflow:global_step/sec: 7.8949
INFO:tensorflow:loss = 0.0040621, step = 302
INFO:tensorflow:global_step/sec: 7.56308
INFO:tensorflow:loss = 0.00266331, step = 402
INFO:tensorflow:global_step/sec: 7.86538
INFO:tensorflow:loss = 0.00194617, step = 502
INFO:tensorflow:global_step/sec: 7.2045
INFO:tensorflow:loss = 0.00151635, step =

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x125773850>, 'hidden_units': [1024, 512, 256, 128, 99], 'feature_columns': (_RealValuedColumn(column_name='', dimension=192, default_value=None, dtype=tf.float32, normalizer=None),), 'embedding_lr_multipliers': None, 'optimizer': None, 'dropout': None, 'gradient_clip_norm': None, 'activation_fn': <function relu at 0x11d9d0050>, 'input_layer_min_slice_size': None})

In [12]:
accuracy_score = classifier.evaluate(x=x_test, y=y_test)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-03-27-19:57:59
INFO

In [13]:
y = classifier.predict(test)
y_prob = classifier.predict_proba(test)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


In [14]:
test_ids = testing.pop('id')
submission = pd.DataFrame(y_prob, index=test_ids, columns=le.classes_)
submission.to_csv('submission_log_reg.csv')